# Deep Dive into LangChain

In [1]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


### Python-dotenv

In [2]:
import os 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

'sk-proj-dsxvcH7KXpdycO17nObyT3BlbkFJivJ8OhSW43DNJlVp6w1D'

## Chat Models: GPT-3.5 Turbo and GPT-4

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
# output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', max_tokens=100, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=['\n', ')
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

Quantum mechanics is a branch of physics that describes the behavior of particles at the smallest scales, where traditional classical physics principles no longer apply and phenomena such as superposition and entanglement occur.


In [4]:
from langchain.schema import (
    SystemMessage, # corresponds to the OpenAI chat completion API
    AIMessage,     # Assistent message
    HumanMessage   # Human messages or prompt
)

messages = [    
    SystemMessage(content='You are a physicist and respond only in portuguese.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

A mecânica quântica é a teoria física que descreve o comportamento das partículas subatômicas em termos de estados superpostos, interferência e probabilidades.


## Caching LLM responses

### In-Memory cache

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [6]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke about quantum mechanics.'
llm.invoke(prompt)

CPU times: user 1.02 s, sys: 48.7 ms, total: 1.07 s
Wall time: 2.84 s


"\n\nWhy did Schrödinger's cat refuse to eat its food?\n\nBecause it was already in a superposition of being both hungry and full!"

After storing it in cache, the time to run the cell decreases to zero!!!

In [7]:
%%time
llm.invoke(prompt)

CPU times: user 1.75 ms, sys: 294 μs, total: 2.04 ms
Wall time: 2.09 ms


"\n\nWhy did Schrödinger's cat refuse to eat its food?\n\nBecause it was already in a superposition of being both hungry and full!"

### SQLite Caching

(to store in the SQLite caching)

In [8]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain_cache.db"))

prompt = 'Tell me a joke about quantum mechanics.'

# First request (not in cache, takes longer)
llm.invoke(prompt)

# Second request (in cache, takes less time)
llm.invoke(prompt)

"\n\nWhy did Schrödinger's cat refuse to come out of the box?\n\nBecause it was afraid of collapsing the waveform!"

### LLM Streaming

In [9]:
from langchain_openai import ChatOpenAI

# This is without streaming
llm = ChatOpenAI()
prompt = 'Tell me a joke about quantum mechanics.'
print(llm.invoke(prompt).content)

Why was Heisenberg such a terrible lover? 

Because when he had the position, he couldn't get the momentum, and when he had the momentum, he couldn't find the position!


In [10]:
# This is with streaming (piece by piece)
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Why was the quantum physicist always calm during experiments? Because he had a lot of momentum!

### Prompt Templates

* Q&A 
* Phrase completions 
* Generating texts

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


template = ''' 
You are an experience physicist and you are explaining quantum mechanics to a student.
Write a short explanation about {topic}.
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = prompt_template.format(topic='tight binding model')
prompt


' \nYou are an experience physicist and you are explaining quantum mechanics to a student.\nWrite a short explanation about tight binding model.\n'

In [12]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

The tight binding model is a simplified approach used in quantum mechanics to describe the behavior of electrons in a solid material. In this model, we consider the electrons to be tightly bound to the atomic cores, and we focus on the interactions between neighboring atoms.

The model assumes that the electrons can only move within a certain range of energy levels, known as bands, which are determined by the interactions between neighboring atoms. These energy bands can be either filled with electrons or empty, depending on the material and its properties.

By studying the tight binding model, we can gain insights into the electronic structure of materials, such as their conductivity, magnetism, and optical properties. This model has been instrumental in understanding the behavior of electrons in solids and has been used to explain a wide range of phenomena in condensed matter physics.


### ChatPromptTemplates

In [32]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
     SystemMessage(content='You are a physicist and respond only in max 5 topics.'),
     HumanMessagePromptTemplate.from_template('Top {n} most spoken physics subject in {area}.'),
    ]
)

messages = chat_template.format_messages(n=5, area='world')
print(messages)

[SystemMessage(content='You are a physicist and respond only in max 5 topics.'), HumanMessage(content='Top 5 most spoken physics subject in world.')]


In [33]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

1. Mechanics
2. Electromagnetism
3. Thermodynamics
4. Quantum Mechanics
5. Relativity


# Simple Chains: single unit task

In [34]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''
You are a physicist and respond only in max 5 topics.
Top {n} most spoken physics subject in {area}.
'''
prompt_template = PromptTemplate.from_template(template=template)

# LLM constructor
chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    # to add intermediate logs (helpful for debugging)
    # verbose=true
)

# A dictionary is used because more than one var can be passed. Otherwise it could be simply chain.invoke(5, 'world')
output = chain.invoke({'n': 5, 'area': 'world'})
print(output)

{'n': 5, 'area': 'world', 'text': '1. Quantum mechanics\n2. General relativity\n3. Thermodynamics\n4. Electromagnetism\n5. Particle physics'}


In [36]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''
You are a physicist and respond only in max 5 topics.
Top 3 most spoken physics subject in {area}.
'''
prompt_template = PromptTemplate.from_template(template=template)

# LLM constructor
chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    verbose=True
)

In [38]:
area = input('Enter a country: ')
output = chain.invoke(area)
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:

You are a physicist and respond only in max 5 topics.
Top 3 most spoken physics subject in Italy.


> Finished chain.
1. Particle Physics
2. Astrophysics
3. Condensed Matter Physics


# Sequential chains: 

make a series of calls to one or more LLMs. Take the output from one chain and use it as the input to another chain. 

There are two typs of sequential chains. 

1. SimpleSequencialChain: represents a seires of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next 

2. General form of sequential chains



### SimpleSequentialChain

In [13]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain


# Be careful with the temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template=''' 
You are an experience physicist and you are explaining quantum mechanics to a student.
Write a short explanation about {concept}.
'''
)

# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)


# Maybe it is interesting to have 0.5 before and 1.2 after
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

prompt_template2 = PromptTemplate.from_template(
    template="Given the {topic}, write a python code for a very simple case"
)

chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)


# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Linear regression is a statistical method used to model the relationship between a dependent variable and one or more independent variables. In the context of quantum mechanics, linear regression can be used to analyze experimental data and determine the relationship between different physical quantities. By fitting a linear equation to the data points, we can make predictions about the behavior of the system and gain insights into the underlying physics. It is a powerful tool that allows us to quantify and understand the relationships between different variables in quantum systems.
In the context of quantum mechanics, a very simple case where we might use linear regression is to determine the relationship between the energy levels of an electron in a potential well and its quantum number. According to the basic principles of quantum mechanics, the relationship between the energy levels (E) of an electron in a simple one-dimensional poten

In [14]:
# The final response is: 
print(output['output'])

In the context of quantum mechanics, a very simple case where we might use linear regression is to determine the relationship between the energy levels of an electron in a potential well and its quantum number. According to the basic principles of quantum mechanics, the relationship between the energy levels (E) of an electron in a simple one-dimensional potential well and their quantum numbers (n) could be modelled linearly for a specific setup (though, keep in mind, this is a simplified example for explanation purposes).

Since this is a fabricated example for educational purposes, let's consider the energy levels to follow a linear relationship such as \(E = h * n\), where \(E\) is the energy, \(n\) is the quantum number, and \(h\) is a proportionality constant. Let's simulate some data for this situation and use Python's SciPy or NumPy library to perform linear regression.

### Preparation
First, ensure you have the necessary libraries. You can install `numpy` and `matplotlib` via 

# LangChain Agents

### LangChain Agents in Action: Python REPL

In [15]:
# Intented for demonstration/research and should be used with care. This isnt in production yet
# pip install -q langchain_experimental 

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1,100) if n % 13 == 0])');

In [20]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)
agent_executer = create_python_agent(
    llm=llm, 
    # Are essentially functions that agents can use for interacting with ouside world. It can vary from chain involving calculators, searches or another chains
    tool=PythonREPLTool(),
    verbose=True
)

# 
agent_executer.invoke('Calculate the square root of the factorial of 12 and display it with 4 decimal points')



> Entering new AgentExecutor chain...
To calculate the square root of the factorial of 12 and display it with 4 decimal points, I will first calculate the factorial of 12 using the `math.factorial()` function. Then, I will calculate the square root of that result using `math.sqrt()`. Finally, I will format the result to display it with 4 decimal points using the `format()` function.
Action: Python_REPL
Action Input: import math
print(format(math.sqrt(math.factorial(12)), '.4f'))
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [21]:
# Note the output is a dictionary containing two key pairs. 
response = agent_executer.invoke('What  is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [22]:
print(response['input'])

What  is the answer to 5.1 ** 7.3?


In [23]:
print(response['output'])

146306.05007233328
